<a href="https://colab.research.google.com/github/alphym/NLP-fellowship/blob/main/igihe_scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assignment
Based on the above, get the main articles from igihe from February 2022 - present

Steps to do this


1.   Get the links to the main pages from january. Create a list
2.   In each link, get all the links to the main articles
3.   For each article, get the main tag that holds the texts
4.   Get the text and store them in a txt file. The data will be used in week 2
5.   Each article its own txt file. Naming is the date_article_1


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
snap = []
err = []
for month in range(1,11):
    for day in range(1,32):
        link = requests.get(url='http://archive.org/wayback/available?url=igihe.com&timestamp=2022{:02d}{:02d}'.format(month, day))
        try:
            if link.json()['archived_snapshots']['closest']['available']:
              #we store each snapshot link into snap list
                snap.append(link.json()['archived_snapshots']['closest']['url'])
        except KeyError:
            err.append(link.json())
            

In [4]:
len(snap)

1

In [5]:
snap_set  = set(snap)

In [6]:
snap_r = list(snap_set)
len(snap_r)

1

In [7]:
#Fetch all the titles and their link from wayback archivr

links_art = []

for lin in snap_r:
  content = requests.get(lin).content #pass each snapshot link to the requests
  soup =  BeautifulSoup(content, "html.parser") #pass each snapshot content html parser
  articles = soup.find_all('span', class_="homenews-title") #look for all title in the snapshot
  links_art.append(articles)

In [8]:
len(list(snap_r))

1

In [9]:
#Actual link
snapshot_link = []
title_ = []

#Loop all the 184 snapshots
for index, snapshot in enumerate(list(snap_r)):
  #In each title assign it's snapshop prefix link
  for title in links_art[index]:
    snapshot_link.append(snapshot)
    title_.append(str(title.find('a')['href']))

In [10]:
len(snapshot_link)

73

In [11]:
df = pd.DataFrame(list(zip(snapshot_link, title_)), columns=['Prefix', 'Title'])

In [12]:
df.head(5)

,Prefix,Title
0,http://web.archive.org/web/20220205000025/http...,amakuru/u-rwanda/article/huye-abanyamadini-n-a...
1,http://web.archive.org/web/20220205000025/http...,imyidagaduro/article/rutayisire-uherutse-kwinj...
2,http://web.archive.org/web/20220205000025/http...,imyidagaduro/sinema/article/menya-zimwe-muri-f...
3,http://web.archive.org/web/20220205000025/http...,amakuru/u-rwanda/article/iburasirazuba-guverin...
4,http://web.archive.org/web/20220205000025/http...,amakuru/u-rwanda/article/umubyeyi-wateganyirij...


In [13]:
df.describe()

,Prefix,Title
count,73,73
unique,1,71
top,http://web.archive.org/web/20220205000025/http...,imyidagaduro/sinema/article/menya-zimwe-muri-f...
freq,73,2


In [14]:
df.drop_duplicates(['Title'])

,Prefix,Title
0,http://web.archive.org/web/20220205000025/http...,amakuru/u-rwanda/article/huye-abanyamadini-n-a...
1,http://web.archive.org/web/20220205000025/http...,imyidagaduro/article/rutayisire-uherutse-kwinj...
2,http://web.archive.org/web/20220205000025/http...,imyidagaduro/sinema/article/menya-zimwe-muri-f...
3,http://web.archive.org/web/20220205000025/http...,amakuru/u-rwanda/article/iburasirazuba-guverin...
4,http://web.archive.org/web/20220205000025/http...,amakuru/u-rwanda/article/umubyeyi-wateganyirij...
...,...,...
66,http://web.archive.org/web/20220205000025/http...,ubutabera-2047/article/ubushinjacyaha-bwamagan...
69,http://web.archive.org/web/20220205000025/http...,ikoranabuhanga/mobayilo/article/menya-smartpho...
70,http://web.archive.org/web/20220205000025/http...,ikoranabuhanga/article/sosiyete-ya-elon-musk-i...
71,http://web.archive.org/web/20220205000025/http...,abantu/kubaho/urukundo/article/shyaka-olivier-...


In [15]:
link = df['Prefix'][0] + df['Title'][0]

content = requests.get(link).content

raw_content = BeautifulSoup(content, 'html.parser')

In [16]:
body = raw_content.find_all('div', class_="fulltext margintop10") 

text = ''
par_ = BeautifulSoup(str(body)).find_all('p')

for line in par_:
  text = text + '\n' + line.get_text()



In [17]:
def get_text(link):
  content = requests.get(link).content
  raw_content = BeautifulSoup(content, 'html.parser')

  body = raw_content.find_all('div', class_="fulltext margintop10") 

  text = ''
  par_ = BeautifulSoup(str(body)).find_all('p')

  for line in par_:
    text = text + '\n' + line.get_text()

In [18]:
def get_article_text(link):
  content = requests.get(link).content
  body = BeautifulSoup(content, 'html.parser').find_all('div', class_='fulltext margintop10')

  text = ''
  try:
    par_ = BeautifulSoup(str(body)).find_all('p')


    for line in par_:
      text = text + '\n' + line.get_text()
  except AttributeError:
    pass
  return text

In [19]:
articles = []

for index, article in enumerate(df['Title']):
  link =df['Prefix'][index]+article
  txt = get_article_text(link)
  articles.append(txt)

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
def get_all_text_file(index):
  link = df['Prefix'][index] + df['Title'][index]

  content = requests.get(link).content

  raw_content = BeautifulSoup(content, 'html.parser')
  body = raw_content.find_all('div', class_="fulltext margintop10") 

  text = ''
  par_ = BeautifulSoup(str(body)).find_all('p')

  for line in par_:
    text = text + '\n' + line.get_text()
  return text


In [22]:
for index in range(len(df)):
  new_t = get_all_text_file(index)
  y = open('/content/drive/MyDrive/NLP-FELLOWSHIP/WEEK1-NLP/content.txt', 'w+',  encoding='utf-8')
  y.write(new_t+' \n')